In [ ]:
import joblib

# Load the saved model  XBG classifier
model = joblib.load('xgb_churn_model.pkl')

In [30]:
import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import LabelEncoder

# Load trained model
model = joblib.load('xgb_churn_model.pkl')



# Load data
df = pd.read_csv('Telco-Customer-Churn.csv')
df = df[df['TotalCharges'] != ' '].copy()
df['TotalCharges'] = df['TotalCharges'].astype(float)

label_encoders = joblib.load('label_encoders.pkl')

# Drop customerID column if it exists (you already saved it separately)
if 'customerID' in df.columns:
    df.drop('customerID', axis=1, inplace=True)

# Binary encode the target column using mapping (no need for LabelEncoder)
df['Churn'] = df['Churn'].map({'Yes': 1, 'No': 0})

# Apply saved encoders to each object column
for col, le in label_encoders.items():
    if col in df.columns:
        df[col] = le.transform(df[col])  # 🔁 Use transform, not fit_transform

# Select features used for training
X = df.drop(columns=['Churn']) 

# Ensure all data types are numeric
X = X.apply(pd.to_numeric, errors='raise')


# Predict churn probabilities
df['Churn_Probability'] = model.predict_proba(X)[:, 1]

# Add back customer ID for reporting
df['customerID'] = customer_ids


GROSS_MARGIN = 0.65
df['CLV'] = (df['MonthlyCharges'] * GROSS_MARGIN) / df['Churn_Probability']
df['RiskScore'] = df['CLV'] * df['Churn_Probability']
df['RiskSegment'] = pd.qcut(df['RiskScore'], q=3, labels=['Low', 'Medium', 'High'])

intervention_effectiveness = {'High': 0.30, 'Medium': 0.15, 'Low': 0.00}
intervention_cost = {'High': 20, 'Medium': 10, 'Low': 0}

df['RetainedRevenue'] = df.apply(
    lambda row: row['CLV'] * intervention_effectiveness[row['RiskSegment']], axis=1
)
df['InterventionCost'] = df['RiskSegment'].map(intervention_cost).astype(float)

total_retained = df['RetainedRevenue'].sum()
total_cost = df['InterventionCost'].sum()
roi = total_retained / total_cost if total_cost > 0 else 0

# Print Summary
print("==== Retention Strategy ROI Summary ====")
print(f"Total Customers: {len(df)}")
print(f"Total Estimated Retained Revenue ($): {round(total_retained, 2)}")
print(f"Total Intervention Cost ($): {round(total_cost, 2)}")
print(f"Estimated ROI: {round(roi, 2)}x")

# Preview
print(df[['customerID', 'MonthlyCharges', 'Churn_Probability', 'CLV', 'RiskSegment', 'RetainedRevenue', 'InterventionCost']].head())


==== Retention Strategy ROI Summary ====
Total Customers: 7032
Total Estimated Retained Revenue ($): 7731356.49
Total Intervention Cost ($): 70300.0
Estimated ROI: 109.98x
   customerID  MonthlyCharges  Churn_Probability          CLV RiskSegment  \
0  7590-VHVEG           29.85           0.509028    38.116789         Low   
1  5575-GNVDE           56.95           0.014853  2492.237815      Medium   
2  3668-QPYBK           53.85           0.580316    60.316250      Medium   
3  7795-CFOCW           42.30           0.015193  1809.660481         Low   
4  9237-HQITU           70.70           0.692394    66.371140      Medium   

   RetainedRevenue  InterventionCost  
0         0.000000               0.0  
1       373.835672              10.0  
2         9.047438              10.0  
3         0.000000               0.0  
4         9.955671              10.0  
